In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net

# NB201
from nas_201_api import NASBench201API as API

2022-09-28 06:33:12.667894: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='../configs/nas-benchmark/algos/RANDOM.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=1, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='../results/220926_single_cell_group/', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=100, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

74983
Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=74983, save_dir='../results/220926_single_cell_group/', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=../results/220926_single_cell_group, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/nas-benchmark/algos/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 1
print_freq       : 100
rand_seed        : 74983
save_dir         : ../results/220926_single_cell_group/
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
        network.module.random_genotype(True)
#         network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

            network.module.random_genotype(True)
#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg


def search_find_best(xloader, network, n_samples):
    with torch.no_grad():
        network.eval()
        archs, valid_accs = [], []
        # print ('obtain the top-{:} architectures'.format(n_samples))
        loader_iter = iter(xloader)
        for i in range(n_samples):
            arch = network.module.random_genotype(True)
#             arch = network.module.random_genotype_per_cell(True)
            try:
                inputs, targets = next(loader_iter)
            except:
                loader_iter = iter(xloader)
                inputs, targets = next(loader_iter)

            _, logits = network(inputs)
            val_top1, val_top5 = obtain_accuracy(
                logits.cpu().data, targets.data, topk=(1, 5)
            )

            archs.append(arch)
            valid_accs.append(val_top1.item())

        best_idx = np.argmax(valid_accs)
        best_arch, best_valid_acc = archs[best_idx], valid_accs[best_idx]
        return best_arch, best_valid_acc

In [5]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/algos/RANDOM.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=250, current::epoch=0, iter=0.00, type=cosine, T-max=250, eta-min=0

In [6]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    cur_arch, cur_valid_acc = search_find_best(
        valid_loader, network, xargs.select_num
    )
    logger.log(
        "[{:}] find-the-best : {:}, accuracy@1={:.2f}%".format(
            epoch_str, cur_arch, cur_valid_acc
        )
    )
    genotypes[epoch] = cur_arch
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log("\n" + "-" * 200)
logger.log("Pre-searching costs {:.1f} s".format(search_time.sum))
start_time = time.time()
best_arch, best_acc = search_find_best(valid_loader, network, xargs.select_num)
search_time.update(time.time() - start_time)
logger.log(
    "RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.".format(
        best_arch, best_acc, search_time.sum
    )
)
if api is not None:
    logger.log("{:}".format(api.query_by_arch(best_arch, "200")))
logger.close()


[Search the 000-250-th epoch] Time Left: [00:00:00], LR=0.025
*SEARCH* [2022-09-28 06:33:17] [000-250][000/391] Time 2.34 (2.34) Data 0.20 (0.20) Base [Loss 2.323 (2.323)  Prec@1 4.69 (4.69) Prec@5 51.56 (51.56)]
*SEARCH* [2022-09-28 06:33:19] [000-250][100/391] Time 0.02 (0.05) Data 0.00 (0.00) Base [Loss 2.036 (2.183)  Prec@1 29.69 (17.09) Prec@5 68.75 (65.90)]
*SEARCH* [2022-09-28 06:33:22] [000-250][200/391] Time 0.02 (0.04) Data 0.00 (0.00) Base [Loss 2.019 (2.122)  Prec@1 17.19 (19.29) Prec@5 81.25 (70.76)]
*SEARCH* [2022-09-28 06:33:25] [000-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.785 (2.088)  Prec@1 37.50 (20.46) Prec@5 82.81 (73.34)]
*SEARCH* [2022-09-28 06:33:27] [000-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.858 (2.062)  Prec@1 27.50 (21.48) Prec@5 82.50 (74.87)]
[000-250] searching : loss=2.06, accuracy@1=21.48%, accuracy@5=74.87%, time-cost=12.6 s
[000-250] evaluate  : loss=2.17, accuracy@1=22.69%, accuracy@5=78.19%
[000-250] find-t

[004-250] find-the-best : Structure(4 nodes with |none~0|+|none~0|avg_pool_3x3~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_1x1~2|), accuracy@1=47.46%
Find ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/seed-74983-last-info.pth
<<<--->>> The 004-250-th epoch : find the highest validation accuracy : 35.55%.
Find ../results/220926_single_cell_group/checkpoint/seed-74983-best.pth exist, delete is at first before saving
copy the file from ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth into ../results/220926_single_cell_group/checkpoint/seed-74983-best.pth

[Search the 005-250-th epoch] Time Left: [01:03:33], LR=0.02497632074113926
*SEARCH* [2022-09-28 06:34:40] [0

*SEARCH* [2022-09-28 06:35:48] [009-250][000/391] Time 0.33 (0.33) Data 0.29 (0.29) Base [Loss 1.227 (1.227)  Prec@1 54.69 (54.69) Prec@5 92.19 (92.19)]
*SEARCH* [2022-09-28 06:35:50] [009-250][100/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 2.322 (1.317)  Prec@1 12.50 (52.38) Prec@5 43.75 (93.19)]
*SEARCH* [2022-09-28 06:35:52] [009-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.159 (1.316)  Prec@1 48.44 (52.39) Prec@5 100.00 (93.52)]
*SEARCH* [2022-09-28 06:35:55] [009-250][300/391] Time 0.04 (0.03) Data 0.00 (0.00) Base [Loss 1.271 (1.312)  Prec@1 46.88 (52.79) Prec@5 95.31 (93.38)]
*SEARCH* [2022-09-28 06:35:58] [009-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.183 (1.322)  Prec@1 55.00 (52.38) Prec@5 92.50 (92.96)]
[009-250] searching : loss=1.32, accuracy@1=52.38%, accuracy@5=92.96%, time-cost=110.9 s
[009-250] evaluate  : loss=1.53, accuracy@1=47.66%, accuracy@5=90.05%
[009-250] find-the-best : Structure(4 nodes with |skip_connect~0|+|nor_conv

*SEARCH* [2022-09-28 06:37:14] [014-250][000/391] Time 0.33 (0.33) Data 0.29 (0.29) Base [Loss 1.082 (1.082)  Prec@1 59.38 (59.38) Prec@5 95.31 (95.31)]
*SEARCH* [2022-09-28 06:37:17] [014-250][100/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.047 (1.187)  Prec@1 57.81 (56.79) Prec@5 96.88 (95.06)]
*SEARCH* [2022-09-28 06:37:19] [014-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.951 (1.179)  Prec@1 65.62 (57.43) Prec@5 96.88 (94.95)]
*SEARCH* [2022-09-28 06:37:22] [014-250][300/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.883 (1.179)  Prec@1 68.75 (57.35) Prec@5 95.31 (94.99)]
*SEARCH* [2022-09-28 06:37:24] [014-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.076 (1.168)  Prec@1 52.50 (57.88) Prec@5 97.50 (94.98)]
[014-250] searching : loss=1.17, accuracy@1=57.88%, accuracy@5=94.98%, time-cost=168.4 s
[014-250] evaluate  : loss=1.47, accuracy@1=51.05%, accuracy@5=90.58%
[014-250] find-the-best : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_

*SEARCH* [2022-09-28 06:38:39] [019-250][100/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.871 (1.100)  Prec@1 71.88 (60.60) Prec@5 96.88 (95.37)]
*SEARCH* [2022-09-28 06:38:42] [019-250][200/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.870 (1.116)  Prec@1 70.31 (60.12) Prec@5 98.44 (94.94)]
*SEARCH* [2022-09-28 06:38:45] [019-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.140 (1.100)  Prec@1 62.50 (60.54) Prec@5 95.31 (95.20)]
*SEARCH* [2022-09-28 06:38:48] [019-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.094 (1.105)  Prec@1 62.50 (60.33) Prec@5 97.50 (94.92)]
[019-250] searching : loss=1.10, accuracy@1=60.33%, accuracy@5=94.92%, time-cost=223.8 s
[019-250] evaluate  : loss=1.38, accuracy@1=52.76%, accuracy@5=89.91%
[019-250] find-the-best : Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_1x1~0|none~1|+|skip_connect~0|nor_conv_1x1~1|avg_pool_3x3~2|), accuracy@1=67.19%
Find ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth

...

*SEARCH* [2022-09-28 07:35:56] [225-250][000/391] Time 0.36 (0.36) Data 0.33 (0.33) Base [Loss 0.674 (0.674)  Prec@1 84.38 (84.38) Prec@5 96.88 (96.88)]
*SEARCH* [2022-09-28 07:35:59] [225-250][100/391] Time 0.03 (0.04) Data 0.00 (0.00) Base [Loss 0.332 (0.466)  Prec@1 82.81 (84.05) Prec@5 100.00 (98.53)]
*SEARCH* [2022-09-28 07:36:03] [225-250][200/391] Time 0.03 (0.04) Data 0.00 (0.00) Base [Loss 0.332 (0.443)  Prec@1 87.50 (84.86) Prec@5 100.00 (98.52)]
*SEARCH* [2022-09-28 07:36:05] [225-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.205 (0.439)  Prec@1 90.62 (84.96) Prec@5 100.00 (98.87)]
*SEARCH* [2022-09-28 07:36:08] [225-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.575 (0.430)  Prec@1 77.50 (85.30) Prec@5 97.50 (99.05)]
[225-250] searching : loss=0.43, accuracy@1=85.30%, accuracy@5=99.05%, time-cost=2457.0 s
[225-250] evaluate  : loss=0.89, accuracy@1=70.78%, accuracy@5=94.96%
[225-250] find-the-best : Structure(4 nodes with |avg_pool_3x3~0|+|nor_c

*SEARCH* [2022-09-28 07:37:27] [230-250][300/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 0.631 (0.441)  Prec@1 81.25 (84.98) Prec@5 98.44 (98.72)]
*SEARCH* [2022-09-28 07:37:28] [230-250][390/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 2.300 (0.439)  Prec@1 15.00 (84.96) Prec@5 50.00 (98.58)]
[230-250] searching : loss=0.44, accuracy@1=84.96%, accuracy@5=98.58%, time-cost=2507.7 s
[230-250] evaluate  : loss=1.09, accuracy@1=63.46%, accuracy@5=92.06%
[230-250] find-the-best : Structure(4 nodes with |avg_pool_3x3~0|+|skip_connect~0|skip_connect~1|+|nor_conv_3x3~0|none~1|nor_conv_3x3~2|), accuracy@1=81.84%
Find ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/seed-74983-l

[235-250] find-the-best : Structure(4 nodes with |nor_conv_3x3~0|+|avg_pool_3x3~0|skip_connect~1|+|skip_connect~0|none~1|nor_conv_3x3~2|), accuracy@1=87.30%
Find ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/seed-74983-last-info.pth

[Search the 236-250-th epoch] Time Left: [00:03:46], LR=0.001185227985649536
*SEARCH* [2022-09-28 07:39:00] [236-250][000/391] Time 0.37 (0.37) Data 0.33 (0.33) Base [Loss 0.440 (0.440)  Prec@1 82.81 (82.81) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 07:39:03] [236-250][100/391] Time 0.03 (0.04) Data 0.00 (0.00) Base [Loss 0.245 (0.403)  Prec@1 93.75 (86.20) Prec@5 100.00 (99.10)]
*SEARCH* [2022-09-28 07:39:06] [236-250][200/391] Time 0.02 (0.03)

*SEARCH* [2022-09-28 07:40:27] [241-250][000/391] Time 0.37 (0.37) Data 0.35 (0.35) Base [Loss 0.763 (0.763)  Prec@1 81.25 (81.25) Prec@5 98.44 (98.44)]
*SEARCH* [2022-09-28 07:40:30] [241-250][100/391] Time 0.08 (0.03) Data 0.00 (0.00) Base [Loss 0.293 (0.501)  Prec@1 89.06 (82.07) Prec@5 100.00 (96.77)]
*SEARCH* [2022-09-28 07:40:32] [241-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.236 (0.472)  Prec@1 92.19 (83.53) Prec@5 100.00 (97.44)]
*SEARCH* [2022-09-28 07:40:34] [241-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.273 (0.461)  Prec@1 95.31 (83.86) Prec@5 100.00 (97.80)]
*SEARCH* [2022-09-28 07:40:37] [241-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.331 (0.465)  Prec@1 82.50 (83.68) Prec@5 100.00 (97.60)]
[241-250] searching : loss=0.47, accuracy@1=83.68%, accuracy@5=97.60%, time-cost=2631.6 s
[241-250] evaluate  : loss=1.04, accuracy@1=65.27%, accuracy@5=90.84%
[241-250] find-the-best : Structure(4 nodes with |nor_conv_1x1~0|+|nor_

*SEARCH* [2022-09-28 07:41:56] [246-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.570 (0.423)  Prec@1 76.56 (85.25) Prec@5 98.44 (98.27)]
*SEARCH* [2022-09-28 07:41:58] [246-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.626 (0.423)  Prec@1 85.00 (85.33) Prec@5 100.00 (98.24)]
[246-250] searching : loss=0.42, accuracy@1=85.33%, accuracy@5=98.24%, time-cost=2683.0 s
[246-250] evaluate  : loss=0.90, accuracy@1=71.61%, accuracy@5=94.04%
[246-250] find-the-best : Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|skip_connect~1|+|none~0|avg_pool_3x3~1|skip_connect~2|), accuracy@1=85.35%
Find ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/seed-74983-

# Train a found model

In [7]:
print(best_arch)
import copy
best_arch_copy = copy.deepcopy(best_arch)

Structure(4 nodes with |skip_connect~0|+|none~0|avg_pool_3x3~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|)


In [8]:
print(args)
tmp = os.path.join(args.save_dir, "train_best_arch")
args.save_dir = tmp
print(args)

Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=74983, save_dir='../results/220926_single_cell_group/', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)
Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=74983, save_dir='../results/220926_single_cell_group/train_best_arch', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [9]:
logger = prepare_logger(args)

cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=../results/220926_single_cell_group/train_best_arch, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/nas-benchmark/algos/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 1
print_freq       : 100
rand_seed        : 74983
save_dir         : ../results/220926_single_cell_group/train_best_arch
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim=

In [10]:
from custom_search_cells import NAS201SearchCell as SearchCell

network.module.arch_cache = best_arch

print(best_arch)

Structure(4 nodes with |skip_connect~0|+|none~0|avg_pool_3x3~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|)


In [11]:
def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
#         network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [12]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func_one_arch(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-09-28 07:43:00] [000-200][000/098] Time 0.39 (0.39) Data 0.35 (0.35) Base [Loss 2.344 (2.344)  Prec@1 8.98 (8.98) Prec@5 51.95 (51.95)]
*SEARCH* [2022-09-28 07:43:04] [000-200][097/098] Time 0.02 (0.04) Data 0.00 (0.01) Base [Loss 1.372 (1.734)  Prec@1 47.62 (34.82) Prec@5 93.45 (86.22)]
[000-200] searching : loss=1.73, accuracy@1=34.82%, accuracy@5=86.22%, time-cost=4.3 s
[000-200] evaluate  : loss=1.65, accuracy@1=39.72%, accuracy@5=90.59%
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 39.72%.
copy the file from ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-best.pth

[Se

*SEARCH* [2022-09-28 07:43:45] [007-200][000/098] Time 0.39 (0.39) Data 0.35 (0.35) Base [Loss 0.763 (0.763)  Prec@1 71.48 (71.48) Prec@5 98.44 (98.44)]
*SEARCH* [2022-09-28 07:43:49] [007-200][097/098] Time 0.04 (0.05) Data 0.00 (0.01) Base [Loss 0.757 (0.734)  Prec@1 75.60 (73.88) Prec@5 97.62 (98.42)]
[007-200] searching : loss=0.73, accuracy@1=73.88%, accuracy@5=98.42%, time-cost=35.0 s
[007-200] evaluate  : loss=0.84, accuracy@1=71.91%, accuracy@5=97.93%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth
<<<--->>> The 007-200-th epoch : find the highest validation accuracy : 71.91%.
Fi

*SEARCH* [2022-09-28 07:44:38] [015-200][000/098] Time 0.39 (0.39) Data 0.34 (0.34) Base [Loss 0.520 (0.520)  Prec@1 82.42 (82.42) Prec@5 99.61 (99.61)]
*SEARCH* [2022-09-28 07:44:42] [015-200][097/098] Time 0.02 (0.04) Data 0.00 (0.02) Base [Loss 0.511 (0.567)  Prec@1 81.55 (80.34) Prec@5 99.40 (99.09)]
[015-200] searching : loss=0.57, accuracy@1=80.34%, accuracy@5=99.09%, time-cost=71.2 s
[015-200] evaluate  : loss=0.73, accuracy@1=75.35%, accuracy@5=98.36%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth
<<<--->>> The 015-200-th epoch : find the highest validation accuracy : 75.35%.
Fi

*SEARCH* [2022-09-28 07:45:32] [023-200][000/098] Time 0.49 (0.49) Data 0.45 (0.45) Base [Loss 0.539 (0.539)  Prec@1 80.86 (80.86) Prec@5 98.83 (98.83)]
*SEARCH* [2022-09-28 07:45:36] [023-200][097/098] Time 0.03 (0.05) Data 0.00 (0.02) Base [Loss 0.552 (0.507)  Prec@1 79.17 (82.38) Prec@5 98.81 (99.28)]
[023-200] searching : loss=0.51, accuracy@1=82.38%, accuracy@5=99.28%, time-cost=107.3 s
[023-200] evaluate  : loss=0.76, accuracy@1=75.69%, accuracy@5=98.28%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth

[Search the 024-200-th epoch] Time Left: [00:21:07], LR=0.09648882429441258
*SEA

*SEARCH* [2022-09-28 07:46:27] [031-200][000/098] Time 0.40 (0.40) Data 0.35 (0.35) Base [Loss 0.477 (0.477)  Prec@1 83.98 (83.98) Prec@5 99.61 (99.61)]
*SEARCH* [2022-09-28 07:46:31] [031-200][097/098] Time 0.02 (0.05) Data 0.00 (0.01) Base [Loss 0.388 (0.471)  Prec@1 88.10 (83.60) Prec@5 100.00 (99.36)]
[031-200] searching : loss=0.47, accuracy@1=83.60%, accuracy@5=99.36%, time-cost=143.9 s
[031-200] evaluate  : loss=0.71, accuracy@1=77.32%, accuracy@5=98.50%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth

[Search the 032-200-th epoch] Time Left: [00:18:35], LR=0.09381533400219318
*SE

...

[164-200] evaluate  : loss=0.50, accuracy@1=85.87%, accuracy@5=99.29%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth

[Search the 165-200-th epoch] Time Left: [00:04:06], LR=0.007367991782295403
*SEARCH* [2022-09-28 08:01:10] [165-200][000/098] Time 0.38 (0.38) Data 0.35 (0.35) Base [Loss 0.100 (0.100)  Prec@1 96.48 (96.48) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 08:01:14] [165-200][097/098] Time 0.02 (0.05) Data 0.00 (0.02) Base [Loss 0.135 (0.115)  Prec@1 94.05 (96.30) Prec@5 100.00 (99.99)]
[165-200] searching : loss=0.11, accuracy@1=96.30%, accuracy@5=99.99%, time-cost=750.1 s


*SEARCH* [2022-09-28 08:01:55] [172-200][000/098] Time 0.40 (0.40) Data 0.36 (0.36) Base [Loss 0.120 (0.120)  Prec@1 96.48 (96.48) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 08:01:59] [172-200][097/098] Time 0.03 (0.05) Data 0.00 (0.01) Base [Loss 0.120 (0.099)  Prec@1 97.02 (96.85) Prec@5 100.00 (100.00)]
[172-200] searching : loss=0.10, accuracy@1=96.85%, accuracy@5=100.00%, time-cost=781.2 s
[172-200] evaluate  : loss=0.49, accuracy@1=86.36%, accuracy@5=99.35%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth

[Search the 173-200-th epoch] Time Left: [00:03:14], LR=0.00442983616822774

*SEARCH* [2022-09-28 08:02:48] [180-200][000/098] Time 0.38 (0.38) Data 0.35 (0.35) Base [Loss 0.073 (0.073)  Prec@1 97.66 (97.66) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 08:02:52] [180-200][097/098] Time 0.02 (0.05) Data 0.00 (0.02) Base [Loss 0.100 (0.084)  Prec@1 96.43 (97.46) Prec@5 100.00 (99.98)]
[180-200] searching : loss=0.08, accuracy@1=97.46%, accuracy@5=99.98%, time-cost=817.3 s
[180-200] evaluate  : loss=0.47, accuracy@1=86.97%, accuracy@5=99.30%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth
<<<--->>> The 180-200-th epoch : find the highest validation accuracy : 86.97%

[187-200] evaluate  : loss=0.47, accuracy@1=86.76%, accuracy@5=99.36%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth

[Search the 188-200-th epoch] Time Left: [00:01:27], LR=0.000885637463565564
*SEARCH* [2022-09-28 08:03:41] [188-200][000/098] Time 0.41 (0.41) Data 0.39 (0.39) Base [Loss 0.061 (0.061)  Prec@1 98.83 (98.83) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 08:03:46] [188-200][097/098] Time 0.03 (0.05) Data 0.01 (0.01) Base [Loss 0.061 (0.072)  Prec@1 98.81 (97.98) Prec@5 100.00 (100.00)]
[188-200] searching : loss=0.07, accuracy@1=97.98%, accuracy@5=100.00%, time-cost=853.6 

*SEARCH* [2022-09-28 08:04:27] [195-200][000/098] Time 0.42 (0.42) Data 0.38 (0.38) Base [Loss 0.055 (0.055)  Prec@1 98.83 (98.83) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 08:04:31] [195-200][097/098] Time 0.03 (0.05) Data 0.00 (0.02) Base [Loss 0.073 (0.067)  Prec@1 98.81 (98.22) Prec@5 100.00 (100.00)]
[195-200] searching : loss=0.07, accuracy@1=98.22%, accuracy@5=100.00%, time-cost=884.9 s
[195-200] evaluate  : loss=0.47, accuracy@1=87.23%, accuracy@5=99.37%
Find ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/checkpoint/seed-74983-basic.pth
Find ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220926_single_cell_group/train_best_arch/seed-74983-last-info.pth
<<<--->>> The 195-200-th epoch : find the highest validation accuracy : 87.2